# Evaluate Dynamic Factor Model

This notebook estimates large mixed-frequency (blocked) Dynamic Factor Models following [Bańbura and Modugno, (2014)](https://onlinelibrary.wiley.com/doi/full/10.1002/jae.2306) and [Bok et al. (2018)](https://www.newyorkfed.org/research/staff_reports/sr830), in Python using the `DynamicFactorMQ` class in [statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.dynamic_factor_mq.DynamicFactorMQ.html). GDP in log-differences is the only quarterly variable, taken from the FRED MQ dataset. The other variables are taken from the FRED-MD dataset and transformed using the transformations suggested in the data appendix. The ground truth estimate is GDP from the 2023-02 vintage. 





### Setup

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
from statsmodels.tsa.api import DynamicFactorMQ
import matplotlib.pyplot as plt
# Some useful functions
def inspect_obj(obj):
    return [x for x in dir(obj) if not x.startswith("_")] 

### Loading Data

In [ ]:
data_path = "../../data/FRED/"
# Vintage Data
monthly_vintages = sorted(os.listdir(data_path + "MD"))
quarterly_vintages = sorted(os.listdir(data_path + "QD"))
if monthly_vintages != quarterly_vintages:
    raise Exception("Monthly and quarterly vintage mismatch")

def clean_names(x):
    new_nam = {y: y.replace(" ", "").upper() for y in x.columns}
    return x.rename(columns = new_nam)

# Load Monthly Data
vintage_data = {v[8:15]: dict(
    monthly = clean_names(pd.read_csv(data_path + "MD/" + v, index_col=0, parse_dates=True).to_period(freq="M")), 
    gdp = pd.read_csv(data_path + "QD/" + v, index_col=0, parse_dates=True).to_period(freq="Q")[["GDPC1"]]
) for v in monthly_vintages}

vintage_data.keys()

#### Plotting GDP Data

In [ ]:
# This plots the ground truth data
vintage_data["2023_02"]["gdp"].plot(title="Real GDP Growth, 2023-02 Vintage", figsize=(12, 6))

In [ ]:
# This gets all the GDP estimates
gdp_all = vintage_data["2018_05"]["gdp"].drop("GDPC1", axis=1)
for k, v in vintage_data.items():
    gdp_all = gdp_all.merge(v["gdp"].rename(columns = {"GDPC1": k}), 
                            how='outer', left_index=True, right_index=True)

gdp_all.head()


In [ ]:
# Plotting all estimates
ax = gdp_all.plot(title="GDP Growth Estimate from All Vintages", figsize=(12, 6), linewidth = 0.2)
ax.legend(ncol=7)
plt.show()

In [ ]:
# ax.get_figure().savefig("figures/GDP_all_vintages.png", dpi = 300, bbox_inches = "tight")
ax.get_figure().savefig("figures/GDP_all_vintages.pdf", format = "pdf", bbox_inches = "tight")

### Labels and Factor Groupings

In [ ]:
# Loading Series Descriptions
series = pd.read_csv(data_path + "FRED-MD Appendix/FRED-MD_updated_appendix.csv", 
                     encoding_errors="ignore")
series.fred = series.fred.transform(lambda y: y.replace(" ", "").upper())
series.groupby("group_name").id.count()

In [ ]:
# Factor Groupings
factors = {l: ['Global', v] for l, v in zip(series.fred, series.group_name)}
factors["GDPC1"] = ['Global', 'Output and Income']

**Factor multiplicities**

The `factor_multiplicities` argument defaults to `1`, but it can be passed a dictionary with keys equal to factor names (from the `factors` argument) and values equal to an integer. Note that the default for each factor is 1, so we only include in this dictionary factors that have multiplicity greater than 1.

In [ ]:
factor_multiplicities = {'Global': 2,
                         'Consumption, Orders, and Inventories': 2,
                         'Money and Credit': 2,
                         'Interest and Exchange Rates': 2}

**Factor orders**

Finally, we need to specify the lag order of the (vector) autoregressions that govern the dynamics of the factors. This is done via the `factor_orders` argument. The `factor_orders` argument also defaults to `1`,

In [ ]:
factor_orders = {'Global': 4, 
                 'Consumption, Orders, and Inventories': 4,  
                 'Housing': 2,
                 'Interest and Exchange Rates': 3, 
                 'Money and Credit': 4,
                 'Output and Income': 2}

In [ ]:
dfm_blocked_2023_02 = DynamicFactorMQ(endog = vintage_data["2023_02"]["monthly"], 
                              endog_quarterly = vintage_data["2023_02"]["gdp"], 
                              factors = factors, 
                              factor_multiplicities = factor_multiplicities, 
                              factor_orders = factor_orders)

In [ ]:
dfm_blocked_2023_02.summary()

In [ ]:
blocked_results_2023_02 = dfm_blocked_2023_02.fit(disp = 10)

In [ ]:
# Another model without separate blocks but 9 global factors
dfm_global_2023_02 = DynamicFactorMQ(endog = vintage_data["2023_02"]["monthly"], 
                                     endog_quarterly = vintage_data["2023_02"]["gdp"], 
                                     factors = 1, 
                                     factor_orders = 3, 
                                     factor_multiplicities = 9) 

In [ ]:
dfm_global_2023_02.summary()

In [ ]:
global_results_2023_02 = dfm_global_2023_02.fit(disp = 10)

### Plotting Factor Estimates

In [ ]:
blocked_results_2023_02.factors.smoothed.plot(title = "Factors from Bocked DFM", figsize = (12, 6))

In [ ]:
global_results_2023_02.factors.smoothed.plot(title = "Factors from Global DFM", figsize = (12, 6))

### Explanatory Power of Factors

In [ ]:
group_counts = series[['fred', 'group_name']].groupby('group_name', sort=False).count()['fred'].cumsum()

def plot_r2_joint(results, group_counts):

    # with sns.color_palette('deep'):
    fig = results.plot_coefficients_of_determination(method='joint', figsize=(14, 3));

    # Add in group labels
    ax = fig.axes[0]
    ax.set_ylim(0, 1.2)
    for i in np.arange(1, len(group_counts), 2):
        start = 0 if i == 0 else group_counts[i - 1]
        end = group_counts[i] + 1
        ax.fill_between(np.arange(start, end) - 0.6, 0, 1.2, color='k', alpha=0.1)
    for i in range(len(group_counts)):
        start = 0 if i == 0 else group_counts[i - 1]
        end = group_counts[i]
        n = end - start
        text = group_counts.index[i]
        if len(text) > n:
            text = text[:n - 3] + '...'

        ax.annotate(text, (start + n / 2, 1.1), ha='center')

    # Add label for GDP
    ax.set_xlim(-1.5, results.data.endog.shape[1] + 0.5)
    ax.annotate('GDP', (results.data.endog.shape[1] - 1.1, 1.05), ha='left', rotation=90)

    fig.tight_layout();

In [ ]:
plot_r2_joint(blocked_results_2023_02, group_counts)

In [ ]:
plot_r2_joint(global_results_2023_02, group_counts)

### DFM Predictions

In [ ]:
def plot_prediction(prediction_results, variables, data):

    # The `predicted_mean` attribute gives the same
    # point forecasts that would have been returned from
    # using the `predict` or `forecast` methods.
    point_predictions = prediction_results.predicted_mean[variables]
    start = point_predictions.index[0]
    latest = data.index[-1]

    # We can use the `conf_int` method to get confidence
    # intervals; here, the 95% confidence interval
    ci = prediction_results.conf_int(alpha=0.05)
    lower = ci[[f'lower {name}' for name in variables]]
    upper = ci[[f'upper {name}' for name in variables]]


    # Plot the forecasts and confidence intervals
    # with sns.color_palette('deep'):
    fig, ax = plt.subplots(figsize=(14, 4))

    # Plot actual data
    data.loc[start:, variables].plot(ax=ax, linestyle='solid',
                                        color=['C0', 'C1', 'C2'])

    # Plot the predictions
    point_predictions.plot(ax=ax, linestyle='dashed',
                            color=['C0', 'C1', 'C2'],
                            legend=False)

    # Confidence intervals
    for name in variables:
        ax.fill_between(ci.index,
                        lower[f'lower {name}'],
                        upper[f'upper {name}'], alpha=0.1)
        
    # Forecast period, set title
    ylim = ax.get_ylim()
    ax.vlines(latest, ylim[0], ylim[1], linewidth=1)
    ax.annotate(r' Forecast $\rightarrow$', (latest, -1.7))
    ax.set(title='In-sample predictions and out-of-sample forecasts, with 95% confidence intervals', ylim=ylim)
    
    fig.tight_layout()

def monthly_index(x):
    x = x.copy()
    x.index = pd.PeriodIndex(x.index, freq="M")
    return x

In [ ]:
blocked_prediction_2023_02 = blocked_results_2023_02.get_prediction(start='2000', end='2024')
global_prediction_2023_02 = global_results_2023_02.get_prediction(start='2000', end='2024')

In [ ]:
plot_prediction(blocked_prediction_2023_02, ['GDPC1'], monthly_index(vintage_data["2023_02"]["gdp"]))

In [ ]:
plot_prediction(global_prediction_2023_02, ['GDPC1'], monthly_index(vintage_data["2023_02"]["gdp"]))

### Model Evaluation

To Evaluate Model Performance, we pedict GDP using models trained on all vintages for a fixed period from 2000 to 2014, each time saving the date GDP was last available.

In [ ]:
all_predictions = {}

In [ ]:
variables = list(factors.keys())

for k, v in vintage_data.items():
    print(k)
    columns = list(v["monthly"].columns)
    dfm_blocked = DynamicFactorMQ(endog = v["monthly"][[x for x in columns if x in variables]], 
                                  endog_quarterly = v["gdp"], 
                                  factors = {k: v for k, v in factors.items() if k in columns + ["GDPC1"]}, 
                                  factor_multiplicities = factor_multiplicities, 
                                  factor_orders = factor_orders)
    blocked_results = dfm_blocked.fit()
    blocked_prediction = blocked_results.get_prediction(start='2000', end='2024')
    
    dfm_global = DynamicFactorMQ(endog = v["monthly"], 
                                 endog_quarterly = v["gdp"], 
                                 factors = 1, 
                                 factor_orders = 3, 
                                 factor_multiplicities = 9)
    global_results = dfm_global.fit()
    global_prediction = global_results.get_prediction(start='2000', end='2024')

    all_predictions[k] = dict(blocked_dfm = blocked_prediction.summary_frame(endog = -1),
                              global_dfm = global_prediction.summary_frame(endog = -1), 
                              gdp = v["gdp"],
                              gdp_latest = str(v["gdp"].index[-1]))


In [ ]:
blocked_dfm_long = pd.concat({k:v["blocked_dfm"].assign(latest_gdp = v["gdp_latest"]) for k, v in all_predictions.items()}) \
          .reset_index(level=0, names = "vintage")

In [ ]:
global_dfm_long = pd.concat({k:v["global_dfm"].assign(latest_gdp = v["gdp_latest"]) for k, v in all_predictions.items()}) \
          .reset_index(level=0, names = "vintage")

In [ ]:
os.makedirs('results', exist_ok=True)
blocked_dfm_long.to_csv("results/blocked_dfm_long.csv", index_label="year_month")
global_dfm_long.to_csv("results/global_dfm_long.csv", index_label="year_month")